In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  1 12:06:06 2020

@author: Naveen
"""

import os
import numpy as np
from keras.datasets import mnist
from selfTraining import selfTrainImageModel, selfTrainTextModel, fullModelImage, fullModelText
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt



In [ ]:
#os.chdir(r'C:\Users\Naveen\Google Drive\Heinz\Coursework\Semester 2\Machine Learning\Project')
#z = gzip.open('./MNIST Data/train-images-idx3-ubyte.gz', 'r')
#image_size = 28
#num_images = 60000
#buf = z.read(image_size * image_size * num_images)
#data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
#xTrain = data.reshape(num_images, image_size, image_size)
#xTrain = xTrain.reshape(60000, 784)
#xTrain = xTrain/255.0
#
#z = gzip.open('./MNIST Data/train-labels-idx1-ubyte.gz','r')
#buf = z.read(num_images)
#yTrain = np.frombuffer(buf, dtype = np.uint8).astype(np.int32)
#del(data)
#del(z)
#del(buf)
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
xTrain = xTrain.reshape(60000, 784)
xTrain = xTrain/255.0
xTest = xTest.reshape(10000, 784)
xTest = xTest/255.0

#xTrain = xTrain[0:5000, :]
#yTrain = yTrain[0:5000]
print(np.unique(yTrain, return_counts=True))

batchSize = 32
maxIter = 5
pThresh = 0.9
labeledSize = 100
selectionType = "Group"

labelSize = [100, 500, 1000, 2000, 5000, 10000, 25000]
selfTrainErr = []
for size in labelSize:
    output = selfTrainImageModel(xTrain, yTrain, batchSize, maxIter, pThresh, size, selectionType)
    wrongLabels = output[1]
    yHat = output[0].predict_classes(xTest, batch_size = batchSize)
    cm = confusion_matrix(yTest, yHat)
    selfTrainErr = np.append(selfTrainErr, 1 - sum(np.diag(cm))/sum(sum(cm)))
    print(wrongLabels)
    
model = fullModelImage(xTrain, yTrain, batchSize)
yHat = model.predict_classes(xTest, batch_size = batchSize)
cm = confusion_matrix(yTest, yHat)
fullErr = 1 - sum(np.diag(cm))/sum(sum(cm))
print(fullErr)
fullErr = np.repeat(fullErr, len(labelSize))
xIdx = list(range(len(labelSize)))
plt.plot(xIdx, selfTrainErr, label = "0% wrong labels")
#plt.plot(xIdx, tempErrorTen, label = "10% wrong labels")
#plt.plot(xIdx, tempErrorTwenty, label = "20% wrong labels")
#plt.plot(xIdx, tempErrorTwentyFive, label = "25% wrong labels")
#plt.plot(xIdx, tempErrorThirtyThree, label = "33% wrong labels")
plt.plot(xIdx, fullErr, label = "Supervised Learning")
plt.xticks(xIdx, labelSize)
plt.xlabel("Number of Labeled Samples")
plt.ylabel("Test Error")
plt.title('Performance of Self-training on MNIST Data')
plt.legend()
plt.savefig('Perf-Self-train-MNIST.png')

def plotRegMNISTModels(layerSize):
    (xTrain, yTrain), (xTest, yTest) = mnist.load_data()
    xTrain = xTrain.reshape(60000, 784)
    xTrain = xTrain/255.0
    xTest = xTest.reshape(10000, 784)
    xTest = xTest/255.0
    
    batchSize = 32
    dropout = [0, 0.05, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5]
    regLambda = 0
    errors = []
    for dopoutPct in dropout:
        model = fullModelImage(xTrain, yTrain, batchSize, layerSize, dopoutPct, regLambda)
        yHat = model.predict_classes(xTest, batch_size = batchSize)
        cm = confusion_matrix(yTest, yHat)
        fullErr = 1 - sum(np.diag(cm))/sum(sum(cm))
        print(fullErr)
        fullErr = np.repeat(fullErr, len(labelSize))
        errors.append(fullErr)
    xIdx = list(range(len(labelSize)))
    for i in range(len(errors)):
        aStr = 'Dropout Pct - ' + str(int(dropout[i] * 100))
        plt.plot(xIdx, errors[i], label = aStr)
    plt.xlabel("Dropout Percentage")
    plt.ylabel("Test Error")
    plt.title('Self-training on MNIST Data - Dropout')
    plt.legend()
    
plotRegMNISTModels(100)

def selfTrainMNIST(pThresh, selectionType, labelSize, induceError, errorDenom, layerSize,
                   dopoutPct, regLambda):
    #1. Label Sizes
    #2. Probabilities
    #3. Wrong Label Percent
    #4. Model hidden layer size
    (xTrain, yTrain), (xTest, yTest) = mnist.load_data()
    xTrain = xTrain.reshape(60000, 784)
    xTrain = xTrain/255.0
    xTest = xTest.reshape(10000, 784)
    xTest = xTest/255.0
    
    #xTrain = xTrain[0:5000, :]
    #yTrain = yTrain[0:5000]
    print(np.unique(yTrain, return_counts=True))
    
    batchSize = 32
    maxIter = 5
    
    selfTrainErr = []
    for size in labelSize:
        output = selfTrainImageModel(xTrain, yTrain, batchSize, maxIter, pThresh, size, selectionType, 
                                     induceError, errorDenom, layerSize, dopoutPct, regLambda)
        wrongLabels = output[1]
        yHat = output[0].predict_classes(xTest, batch_size = batchSize)
        cm = confusion_matrix(yTest, yHat)
        selfTrainErr = np.append(selfTrainErr, 1 - sum(np.diag(cm))/sum(sum(cm)))
        print('Denom ', errorDenom, 'Label Size ', size, wrongLabels)
        
    model = fullModelImage(xTrain, yTrain, batchSize, layerSize, dopoutPct, regLambda)
    yHat = model.predict_classes(xTest, batch_size = batchSize)
    cm = confusion_matrix(yTest, yHat)
    fullErr = 1 - sum(np.diag(cm))/sum(sum(cm))
    print(fullErr)
    fullErr = np.repeat(fullErr, len(labelSize))
    return([selfTrainErr, fullErr])
    
def plotBaseModels(pThresh, selectionType, labelSize, induceError, errorDenom, pltName):
    layerSize = 100
    base512errors = selfTrainMNIST(pThresh, selectionType, labelSize, induceError, errorDenom, 
                                   layerSize, 0.25, 0)
    layerSize = 100
    base100errors = selfTrainMNIST(pThresh, selectionType, labelSize, induceError, errorDenom, 
                                   layerSize, 0, 0.01)
    xIdx = list(range(len(labelSize)))
    plt.plot(xIdx, base512errors[0], label = "Dropout - 25%")
    plt.plot(xIdx, base512errors[1], label = "Supervised Learning - Dropout - 25%")
    plt.plot(xIdx, base100errors[0], label = "L2 Reg - lambda 0.01")
    plt.plot(xIdx, base100errors[1], label = "Supervised Learning - L2 Reg - lambda 0.01")
    plt.xticks(xIdx, labelSize)
    plt.xlabel("Number of Labeled Samples")
    plt.ylabel("Test Error")
    plt.title('Performance of Self-training on MNIST Data')
    plt.legend()
    plt.savefig(pltName + '_Self_train_MNIST.png')
    
def plotWrongLabelModels(pThresh, selectionType, labelSize, induceError, errorDenom, pltName):
    layerSize = 100
    wrongLabelErrors = []
    errors = selfTrainMNIST(pThresh, selectionType, labelSize, False, 0, layerSize)
    wrongLabelErrors.append([errors[0].tolist()])
    if induceError == True:
        for denom in errorDenom:
            errors = selfTrainMNIST(pThresh, selectionType, labelSize, induceError, denom, layerSize)
            wrongLabelErrors.append([errors[0].tolist()])
            #wrongLabelErrors = np.append(wrongLabelErrors, errors[0])
    print(wrongLabelErrors)
    print(errors[1])
    xIdx = list(range(len(labelSize)))
    plt.plot(xIdx, wrongLabelErrors[0][0], label = "0% wrong labels")
    if induceError == True:
        for i in range(len(errorDenom)):
            label = str(int(100/errorDenom[i]))
            plt.plot(xIdx, wrongLabelErrors[1 + i][0], label = label + "% wrong labels")
    plt.plot(xIdx, errors[1], label = "Supervised Learning")
    plt.xticks(xIdx, labelSize)
    plt.xlabel("Number of Labeled Samples")
    plt.ylabel("Test Error")
    plt.title('Performance of Self-training on MNIST Data')
    plt.legend()
    plt.savefig(pltName + '_Self_train_MNIST.png')
    return([wrongLabelErrors, errors[1]])
    
pThresh = 0.9
selectionType = "Group"
labelSize = [100, 500, 1000, 2000, 5000, 10000, 25000]
induceError = False
errorDenom = 10
pltName = 'RegvsDropout'
plotBaseModels(pThresh, selectionType, labelSize, induceError, errorDenom, pltName)

induceError = True
labelSize = [100, 500, 1000, 2000]
errorDenom = [20, 10, 5, 4, 2]
pltName = 'Wrong_Labels'
wrongLabelErrors = plotWrongLabelModels(pThresh, selectionType, labelSize, induceError, 
                                        errorDenom, pltName)

In [3]:
#############################################################
###################### IMDB using LSTM ######################
#############################################################
from keras.preprocessing import sequence
from keras.datasets import imdb

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(xTrain, yTrain), (xTest, yTest) = imdb.load_data(num_words = max_features)
print(len(xTrain), 'train sequences')
print(len(xTest), 'test sequences')

print('Pad sequences (samples x time)')
xTrain = sequence.pad_sequences(xTrain, maxlen=maxlen)
xTest = sequence.pad_sequences(xTest, maxlen=maxlen)
print('xTrain shape:', xTrain.shape)
print('xTest shape:', xTest.shape)

#model = fullModelText(xTrain, yTrain, batchSize)
#score, acc = model.evaluate(xTest, yTest, batch_size = batch_size)
#print('Test score:', score)
#print('Test accuracy:', acc)


batchSize = 32
maxIter = 5
pThresh = 0.95
labeledSize = 100
selectionType = "Group"
dataType = 'text'
maxFeatures = 20000

dropout = 0.2
regLambda = 0
labelSize = [1000, 2000, 5000, 10000, 20000]
selfTrainErr = []
for size in labelSize:
    output = selfTrainTextModel(xTrain, yTrain, batchSize, maxIter, pThresh, size, selectionType,
                                maxFeatures, dropout, regLambda)
    wrongLabels = output[1]
    yHat = output[0].predict_classes(xTest, batch_size = batchSize)
    cm = confusion_matrix(yTest, yHat)
    selfTrainErr = np.append(selfTrainErr, 1 - sum(np.diag(cm))/sum(sum(cm)))
    print(wrongLabels)
    
model = fullModelText(xTrain, yTrain, batchSize, dropout, regLambda)
yHat = model.predict_classes(xTest, batch_size = batchSize)
cm = confusion_matrix(yTest, yHat)
fullErr = 1 - sum(np.diag(cm))/sum(sum(cm))
print(fullErr)
fullErr = np.repeat(fullErr, len(labelSize))
xIdx = list(range(len(labelSize)))
plt.plot(xIdx, selfTrainErr, color = 'red', label = "Self Training")
plt.plot(xIdx, fullErr, color = "blue", label = "Supervised Learning")
plt.xticks(xIdx, labelSize)
plt.xlabel("Number of Labeled Samples")
plt.ylabel("Test Error")
plt.title('Self-training on IMDB Data')
plt.legend()
plt.savefig('Self-train-IMDB.png')

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
xTrain shape: (25000, 80)
xTest shape: (25000, 80)


TypeError: fullModelText() missing 2 required positional arguments: 'dropout' and 'regLambda'